In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

In [ ]:
# %%bash
# pip3 install gymnasium[classic_control]

In [ ]:
%%bash
pip3 install gymnasium[accept-rom-license,atari]

In [ ]:
pip install renderlab

In [ ]:
import gymnasium as gym
import cv2
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import renderlab as rl

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

env = gym.make("ALE/SpaceInvaders-v5")

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
state, info = env.reset()

In [ ]:
def preprocess(image):
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  resized_image = cv2.resize(gray_image, (84, 110))
  cropped_image = resized_image[13:97, :]
  return cropped_image

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

Noisy Networks with factorized Gaussian noise in accordance with the paper "Noisy Networks for Exploration" Fortunato et al. (2019)

In [ ]:
class NoisyLinear(nn.Module):
    """
    NoisyNet layer with factorized Gaussian noise.
    """
    def __init__(self, in_features, out_features, sigma_zero=0.4, bias=True):
        super(NoisyLinear, self).__init__()

        # Initialization
        self.in_features = in_features
        self.out_features = out_features
        sigma_init = sigma_zero / math.sqrt(in_features)

        # Initialize weights
        self.mu_weight = nn.Parameter(torch.Tensor(out_features, in_features))
        self.sigma_weight = nn.Parameter(torch.full((out_features, in_features), sigma_init))

        # Initialize bias if bias is enabled
        self.bias = bias
        if bias:
            self.mu_bias = nn.Parameter(torch.Tensor(out_features))
            self.sigma_bias = nn.Parameter(torch.full((out_features,), sigma_init))

        # Initialitize the values
        self.reset_parameters()

    def reset_parameters(self):
        std = math.sqrt(1 / self.in_features)
        nn.init.uniform_(self.mu_weight, -std, std)
        nn.init.uniform_(self.mu_bias, -std, std)

    def forward(self, input):
        # Generate noise
        epsilon_input = torch.randn((1, self.in_features))
        epsilon_output = torch.randn((self.out_features, 1))

        # Function used in "Noisy Networks for Exploration" (factorized gaussian noise case)
        eps_in = torch.sign(epsilon_input) * torch.sqrt(torch.abs(epsilon_input))
        eps_out = torch.sign(epsilon_output) * torch.sqrt(torch.abs(epsilon_output))
        noise_v = torch.mul(eps_in, eps_out)

        # Add noise to bias if bias is enabled
        if self.bias:
            bias = self.mu_bias + self.sigma_bias * eps_out.t()

        # Apply noisy linear transformation
        return F.linear(input, self.mu_weight + self.sigma_weight * noise_v, bias)

In [ ]:
class DQN(nn.Module):

    def __init__(self, in_channels, n_actions):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.fc1 = NoisyLinear(5184, 256)
        self.fc2 = NoisyLinear(256, n_actions)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x, start_dim=1)
        print(x.shape)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [ ]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4
N_FRAMES = 1

# Get number of actions from gym action space
n_actions = env.action_space.n
# Get the number of state observations
state, info = env.reset()
# n_observations = len(state)

policy_net = DQN(N_FRAMES, n_actions).to(device)
target_net = DQN(N_FRAMES, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return the largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            a = policy_net(state)
            return a.max(1).indices.view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)


episode_durations = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

def plot_scores(scores, show_result=False):
    plt.figure(1)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Score')
    plt.plot(scores)
    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state).reshape(BATCH_SIZE, 1, 84, 84)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1).values
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [ ]:
if torch.cuda.is_available():
    num_episodes = 600
else:
    num_episodes = 50

scores = []

for i_episode in range(num_episodes):
    # Initialize the environment and get its state
    state, info = env.reset()
    score = 0
    state = preprocess(state)
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0).unsqueeze(0)
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        observation = preprocess(observation)
        reward = torch.tensor([reward], device=device)
        score += reward.detach().cpu()
        done = terminated or truncated
        if terminated:
            next_state = None
            scores.append(score)
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)
        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # DOUBLE Q LEARNING
        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            plot_scores(scores)
            # plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

torch.Size([1, 5184])
torch.Size([256])
torch.Size([1, 256])
torch.Size([6])
torch.Size([1, 6])
torch.Size([1, 5184])
torch.Size([256])
torch.Size([1, 256])
torch.Size([6])
torch.Size([1, 6])
torch.Size([1, 5184])
torch.Size([256])
torch.Size([1, 256])
torch.Size([6])
torch.Size([1, 6])
torch.Size([1, 5184])
torch.Size([256])
torch.Size([1, 256])
torch.Size([6])
torch.Size([1, 6])
torch.Size([1, 5184])
torch.Size([256])
torch.Size([1, 256])
torch.Size([6])
torch.Size([1, 6])
torch.Size([1, 5184])
torch.Size([256])
torch.Size([1, 256])
torch.Size([6])
torch.Size([1, 6])
torch.Size([1, 5184])
torch.Size([256])
torch.Size([1, 256])
torch.Size([6])
torch.Size([1, 6])
torch.Size([1, 5184])
torch.Size([256])
torch.Size([1, 256])
torch.Size([6])
torch.Size([1, 6])
torch.Size([1, 5184])
torch.Size([256])
torch.Size([1, 256])
torch.Size([6])
torch.Size([1, 6])
torch.Size([1, 5184])
torch.Size([256])
torch.Size([1, 256])
torch.Size([6])
torch.Size([1, 6])
torch.Size([1, 5184])
torch.Size([256])


KeyboardInterrupt: 

In [ ]:
env = gym.make("ALE/SpaceInvaders-v5", render_mode="rgb_array")
env = rl.RenderFrame(env, "./output")

state, info = env.reset()

state = preprocess(state)
state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0).unsqueeze(0)
for t in count():
    action = select_action(state)
    observation, reward, terminated, truncated, _ = env.step(action.item())

    observation = preprocess(observation)
    done = terminated or truncated
    if terminated:
        next_state = None
        break
    else:
        next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0).unsqueeze(0)
    state = next_state
env.play()


Moviepy - Building video temp-{start}.mp4.
Moviepy - Writing video temp-{start}.mp4



Moviepy - Done !
Moviepy - video ready temp-{start}.mp4
